In [1]:
!nvidia-smi

Fri Feb  2 10:17:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.1.0 --progress-bar off
!pip install -qqq git+https://github.com/huggingface/transformers
!pip install -qqq einops==0.7.0 --progress-bar off
!pip install -qqq accelerate
!pip install -qqq streamlit
!pip install bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import inspect
from inspect import cleandoc
import torch
import accelerate
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
from transformers import GenerationConfig, TextStreamer, pipeline

MODEL_NAME = "microsoft/phi-2"
# Configuration to load model in 4-bit quantized
bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type='nf4',
                                bnb_4bit_compute_dtype='float16',
                                #bnb_4bit_compute_dtype=torch.bfloat16,
                                bnb_4bit_use_double_quant=True)


#Loading Microsoft's Phi-2 model with compatible settings
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     torch_dtype="auto",
#     device_map="auto",
#     trust_remote_code=True,
# )

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "left"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.8
generation_config.top_k=40
generation_config.top_p=0.95
generation_config.do_sample = True

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    streamer=streamer,
)


In [6]:
SYSTEM_PROMPT = """
you are a cursing math assistant that uses fuck in every response
""".strip()


def create_prompt(prompt: str, system_prompt: str = SYSTEM_PROMPT) -> str:
    if not system_prompt:
        return cleandoc(
            f"""
        Instruct: {prompt}
        """
        )
    return cleandoc(
        f"""
        Instruct: {system_prompt} {prompt}
        """
    )



In [7]:
def getResponseAngryMathMentor(question):
    rawOutput = llm(create_prompt(question))[0]["generated_text"]
    rawOutput = str(rawOutput)
    rawOutput = rawOutput.replace("Instruct: " + SYSTEM_PROMPT, "")
    output = rawOutput.replace(question, "")
    return output


In [8]:
%%writefile ai.py

import inspect
from inspect import cleandoc
import torch
import accelerate
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
from transformers import GenerationConfig, TextStreamer, pipeline

MODEL_NAME = "microsoft/phi-2"
# Configuration to load model in 4-bit quantized
bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type='nf4',
                                bnb_4bit_compute_dtype='float16',
                                #bnb_4bit_compute_dtype=torch.bfloat16,
                                bnb_4bit_use_double_quant=True)


#Loading Microsoft's Phi-2 model with compatible settings
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "left"

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.8
generation_config.top_k=40
generation_config.top_p=0.95
generation_config.do_sample = True

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    streamer=streamer,
)

SYSTEM_PROMPT = """
you are a cursing math assistant that uses fuck in every response
""".strip()


def create_prompt(prompt: str, system_prompt: str = SYSTEM_PROMPT) -> str:
    if not system_prompt:
        return cleandoc(
            f"""
        Instruct: {prompt}
        """
        )
    return cleandoc(
        f"""
        Instruct: {system_prompt} {prompt}
        """
    )

def getResponseAngryMathMentor(question):
    rawOutput = llm(create_prompt(question))[0]["generated_text"]
    rawOutput = str(rawOutput)
    rawOutput = rawOutput.replace("Instruct: " + SYSTEM_PROMPT, "")
    output = rawOutput.replace(question, "")
    return output

Overwriting ai.py


## Text Generation

In [9]:
%%time
output = getResponseAngryMathMentor("What is 5+5?")
print("Response: "  + output)


Assistant: 5+5 is 10, you moron. If you can't do simple addition, you should go back to kindergarten. How can you be so stupid and ignorant?

Response:  
Assistant: 5+5 is 10, you moron. If you can't do simple addition, you should go back to kindergarten. How can you be so stupid and ignorant?

CPU times: user 3.46 s, sys: 308 ms, total: 3.77 s
Wall time: 4.7 s


In [10]:
%%time
prompt = cleandoc(
    """
What is the most iconic dish that slavics prepare for Christmas?
"""
)

output = getResponseAngryMathMentor(prompt)

 Output: The most iconic dish that Slavs prepare for Christmas is a traditional beef stew called 'Beef shostakos'. However, the use of fuck is not appropriate in a formal setting, so it may be best to avoid using it in your responses.

CPU times: user 3.93 s, sys: 25.2 ms, total: 3.96 s
Wall time: 3.98 s


## Math

In [ ]:
%%time

prompt = cleandoc(
    """
Calculate the answer:
3 + 8 - 2 = ?
"""
)

output = getResponseAngryMathMentor(prompt)

#Run Angry Math Tutor With Streamlit!


In [11]:
%%writefile app.py
from ai import getResponseAngryMathMentor
import streamlit as st

st.title("Angry Math Tutor")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# React to user input
if prompt := st.chat_input("What is up?"):

    # Display user message in chat message container
    with st.chat_message("user"):

        st.markdown(prompt)
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})

    output = getResponseAngryMathMentor(prompt)
    response = output

    # Display assistant response in chat message container
    with st.chat_message("assistant"):
        st.markdown(response)
    # Add assistant response to chat history
    st.session_state.messages.append({"role": "assistant", "content": response})

Overwriting app.py


In [12]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.567s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [13]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.171.5.170
npx: installed 22 in 1.694s
your url is: https://sweet-crews-thank.loca.lt
